Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns 12 dollars in profit. Soccer trophies are similar (a brass ball, a wood base, and an engraved plaque) except that a brass soccer ball is on top, and the unit profit is only 9 dollars. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?

**Base Version**

In [6]:
# always specify which packages you're going to use
using JuMP, Clp

#create a new model object
m = Model()

# we need variables for football trophies and soccer trophies
# format is (<model name>, <variable name>). we can optionally
# include bounds on each variable.
@variable(m, ft >= 0)
@variable(m, st >= 0)

# objective is to maximize profit
# format is (<model name>, <Max or Min>, <algebraic function>)
@objective(m, Max, 12*ft + 9*st)

# constraint on the wood available
# format is (<model name>, <constraint name>, <algebraic constraint>)
@constraint(m, wood_con, 4ft + 2st <= 4800)

#constraint on the plaques available
@constraint(m, plaque_con, ft + st <= 1750)

# constraints on brass footballs, soccerballs available
@constraint(m, brass_football_con, ft <= 1000)
@constraint(m, brass_soccerball_con, st <= 1500)
; 
# like Matlab, a ";" (semicolon) supresses output

In [2]:
println("Time to solve this model using Clp: ")

# specify the solver you want to use to solve Model m
set_optimizer(m, Clp.Optimizer)

# use the @time macro to measure the amount of time it takes to solve m
@time(optimize!(m))

println("Build ", value(ft), " football trophies.")
println("Build ", value(st), " soccer trophies.")
println("Total profit will be \$", objective_value(m))

Time to solve this model using Clp: 
  1.052474 seconds (2.31 M allocations: 153.052 MiB, 2.79% gc time, 97.24% compilation time)
Build 650.0 football trophies.
Build 1100.0 soccer trophies.
Total profit will be $17700.0
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


In [4]:
using SCS

println("Time to solve this model using SCS: ")
set_optimizer(m, SCS.Optimizer)
@time(optimize!(m))

println("Build ", value(ft), " football trophies.")
println("Build ", value(st), " soccer trophies.")
println("Total profit will be \$", objective_value(m))

# some solvers (including SCS), output a lot of information along with the solution.
# it can be helpful to explicitly print some desired solution components, as we've done here.

Time to solve this model using SCS: 
------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2, constraints m: 6
cones: 	  l: linear vars: 6
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 8, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.80e+03  1.20e+01  3.13e+04 -1.57e+04  1.00e-01  2.35e-03 
   250| 6.49e+00  3.48e-02  9.58e+00 -1.77e+04  5.33e-03  4.71e-03 
   300| 3.73e-01  1.00e-03  1.19e+00 -1.77e

**Modular Version**

We can improve our code's modifiability and reusability by separating the data from the structure.

In [7]:
# An Array in Julia is a list of elements
trophy_types = [:football, :soccer] # these are the possible trophy types

# this array is of type "Symbol." Putting a colon in front of an element makes it a "Symbol" type.
# typically Symbol arrays are easier to implement in JuMP than other types (such as string, integer, etc.)

# A Dictionary in Julia maps a key (any type) to an element (any type)
wood_req = Dict(:football => 4, :soccer => 2) # how much wood each trophy type will use

# in this Dictionary, we are mapping keys of Symbol type (:football) to elements of integer type (4)

plaque_req = Dict(:football => 1, :soccer => 1) # how many plaques each trophy type will use

profit = Dict( :football => 12, :soccer => 9) # profit produced by each trophy type

# we are told the amount of each resource we have available
wood_avail = 4800
plaques_avail = 1750
football_avail = 1000
soccer_avail = 1500;     # semicolons supress output

In [8]:
# always specify which packages you're going to use
using JuMP, Clp

#create a new model object, specifying the solver
m = Model(Clp.Optimizer)

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# maximize profit by summing (profit/trophy * # trophies) for each type
@objective(m, Max, sum(profit[i] * trophy[i] for i in trophy_types) )   

@constraint(m, sum(wood_req[i] * trophy[i] for i in trophy_types) <= wood_avail) # use only available wood
@constraint(m, sum(plaque_req[i] * trophy[i] for i in trophy_types) <= plaques_avail) # use only available plaques
@constraint(m, trophy[:football] <= football_avail)  # use only available brass footballs
@constraint(m, trophy[:soccer] <= soccer_avail)  # use only available brass soccer balls

status = optimize!(m) # solve instance of model

println(value.(trophy)) # print values of trophy variables 
# note the output is quite ugly. we'll see how to make it look nicer soon.
println("Total profit will be \$", objective_value(m))
println("We will use ", value(sum(wood_req[i] * trophy[i] for i in trophy_types) ), " board feet of wood")
println("We will use ", value(sum(plaque_req[i] * trophy[i] for i in trophy_types) ), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


In [9]:
# create expressions for the total amount of wood and plaques used over all trophies
# syntax is (<model name>, <expression name>, <algebraic function>)
@expression(m, tot_wood, sum(wood_req[i] * trophy[i] for i in trophy_types) )
@expression(m, tot_plaques, sum(plaque_req[i] * trophy[i] for i in trophy_types) )
# create an expression for the total amount of profit over all trohpies
@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

# constraints become much simpler
@constraint(m, tot_wood <= wood_avail)
@constraint(m, tot_plaques <= plaques_avail)
@constraint(m, trophy[:football] <= football_avail)
@constraint(m, trophy[:soccer] <= soccer_avail)

# the objective is also very simple
@objective(m, Max, tot_profit)

# solve the instance of the model 
status = optimize!(m)

# print interesting solution components, including expression values
println(value.(trophy))
println("Total profit will be \$", objective_value(m))
println("We will use ", value(tot_wood), " board feet of wood")
println("We will use ", value(tot_plaques), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-6) rows, 2 (0) columns and 4 (-8) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


**Compact Version**

Like the modular version, we separate the data from the model structure. Now we also introduce the "NamedArray" object, which is a nice way to format arrays of data.

In [11]:
trophy_types = [:football, :soccer] # these are the possible trophy types

resources = [:wood, :plaques, :brass_football, :brass_soccer] # what goes into each trophy

profit = Dict( zip(trophy_types, [12,9] ) ) # profit produced by each trophy type

resource_avail = Dict( zip(resources, [4800, 1750, 1000, 1500] ) ); # amount of each type of resource available

# we use the NamedArrays package (you'll need to Pkg.add it first)
using NamedArrays

# create a matrix (Array) of the "recipe" for each trophy type. 
# each row is a trophy type, each column is a resource (should be ordered the same as the resources array).
# we read this as: "trophy type 1 (:football) requires 4 of resource 1 (:wood), 1 of resource 2 (:plaques),
# 1 of resource 3 (:brass_football), and 0 of resouce 4 (:brass_soccer)."
# row 2 is similar, but for trophy type :soccer.
trophy_resource_matrix = [4 1 1 0
                            2 1 0 1]

# create NamedArray that contains info on how much of each resource each trophy uses.
# syntax is (<"recipe" matrix>, (<row indices>,<column indices>),(<row name>,<column name>))
trophy_resource_NA = NamedArray(trophy_resource_matrix, (trophy_types, resources), ("type","resource"))

# check out the output to see how NamedArrays are structured:

2×4 Named Matrix{Int64}
type ╲ resource │           wood  …    brass_soccer
────────────────┼──────────────────────────────────
football        │              4  …               0
soccer          │              2  …               1

In [12]:
# always specify which packages you're going to use
using JuMP, Clp

#create a new model object, specifying the solver
m = Model(Clp.Optimizer)

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# use an expression object to calculate the total profit
@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

# our trophy/resource NamedArray allows us to create a Dictionary of constraints.
# indices are resources, and elements are constraint objects.
@constraint(m, constr[i in resources], sum(trophy_resource_NA[t, i] * trophy[t] for t in trophy_types) <= resource_avail[i] )

# our objective is to maximize the total profit
@objective(m, Max, tot_profit)

# solve the instance of the problem
optimize!(m)

# display solution information
println(value.(trophy))
println("Total profit will be \$", objective_value(m))


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00
